In [10]:
import numpy as np

# ------------------------------
# Parametry zadania
# ------------------------------
C = 0.15
h = np.array([0.352, 0.763, 0.518])
I = np.array([1.19, 1.87, 3.19])


def T(p): #T(p) function from the task
    val = 0.0
    for k in range(3):
        D_k = I[k] + np.sum(p[np.arange(3) != k] * h[np.arange(3) != k])
        numerator = 1.0 + (p[k] * h[k]) / D_k
        val += np.log2(numerator)
    return val


def u(p): #objective function
    return T(p) - C * np.sum(p)


def grad_T(p): #T(p) gradient

    grad = np.zeros(3)
    for k in range(3):
        # T_k = log2( 1 + p_k * h_k / D_k )
        #derivatives
        D_k = I[k] + np.sum(p[np.arange(3) != k] * h[np.arange(3) != k])
        
        for j in range(3):
            if j == k:
                # d/dp_k: 1/ln(2) * [h_k / (D_k + p_k*h_k)]
                grad[j] += (h[k] / (D_k + p[k]*h[k])) / np.log(2)
            else:
                # d/dp_j: -1/ln(2) * [p_k*h_k*h_j / (D_k*(D_k + p_k*h_k))]
                grad[j] += - (p[k]*h[k]*h[j]) / (D_k*(D_k + p[k]*h[k])) / np.log(2)
    return grad

# ------------------------------
# Gradient u(p) = grad_T(p) - C
# ------------------------------
def grad_u(p):
    gT = grad_T(p)
    # odjąć wektor (C, C, C)
    return gT - C

# ------------------------------
# Projekcja na zbiór {p >= 0, sum p <= 1}
# ------------------------------
def project_simplex(p):
    """
    Rzut Euklidesowy na zbiór { p_i >= 0, sum_i p_i <= 1 }.
    """
    # Najpierw obcinamy do zera:
    p_clipped = np.maximum(p, 0.0)
    s = np.sum(p_clipped)
    if s <= 1.0:
        # Jest już w środku
        return p_clipped
    else:
        # Musimy zrzutować na sum = 1.
        # Standardowy trick: sortujemy i znajdujemy próg.
        # Ewentualnie można użyć szybkich bibliotek, tu wersja "ręczna".
        u_sorted = np.sort(p_clipped)[::-1]  #descending
        cssv = np.cumsum(u_sorted)
        rho = 0
        for i in range(len(u_sorted)):
            # warunek: u_sorted[i] - (cssv[i] - 1)/(i+1) > 0
            if u_sorted[i] - (cssv[i] - 1.0)/(i+1) > 0:
                rho = i+1
        theta = (cssv[rho-1] - 1.0) / rho
        w = np.maximum(p_clipped - theta, 0.0)
        return w

# ------------------------------
# Metoda gradientu z projekcją i Armijo
# ------------------------------
def maximize_u_armijo(p0, max_iter=200, tol=1e-6, alpha_init=1.0, beta=0.5, sigma=1e-4):
    """
    p0        -- punkt startowy (np. [0,0,0])
    max_iter  -- maksymalna liczba iteracji
    tol       -- próg, przy którym uznajemy, że gradient jest 'mały'
    alpha_init-- początkowy krok w warunku Armijo
    beta      -- współczynnik zmniejszania kroku (0<beta<1)
    sigma     -- współczynnik w kryterium Armijo (0<sigma<1)
    """
    p = p0.copy()
    for it in range(max_iter):
        g = grad_u(p)
        norm_g = np.linalg.norm(g)
        if norm_g < tol:
            # gradient bardzo mały -> koniec
            break
        
        # Kierunek - w metodzie "maksymalizacji" idziemy w kierunku +g
        d = g
        
        # Szukamy kroku alpha wg Armijo
        alpha = alpha_init
        left = u(p + alpha*d)  # ta wersja jest przed projekcją...
        # Teoretycznie powinniśmy do warunku wstawić zrzutowany punkt,
        # ale często stosuje się "Armijo z uwzględnieniem projekcji".
        # Można zrobić:
        # left = u( project_simplex(p + alpha*d) )
        # W pełnym "projection + Armijo" rzut wykonujemy na każdym kroku testu.
        
        # Prawa strona:
        right = u(p) + sigma * alpha * np.dot(g, d)
        
        # Dopóki warunek nie jest spełniony, zmniejszamy alpha:
        while True:
            p_candidate = project_simplex(p + alpha*d)
            left = u(p_candidate)
            right = u(p) + sigma * alpha * np.dot(g, d)
            if left >= right:
                break
            alpha *= beta
            if alpha < 1e-12:
                # krok za mały, przerywamy
                break
        
        # Aktualizacja p
        p = project_simplex(p + alpha*d)
    
    return p

# ------------------------------
# Uruchomienie
# ------------------------------



In [9]:

# Start np. w środku: (1/3, 1/3, 1/3) lub (0,0,0).
p_start = np.array([1/3, 1/3, 1/3])
    
p_opt = maximize_u_armijo(p_start, max_iter=500)
print("Rozwiązanie p* =", p_opt)
print("Suma p* =", p_opt.sum())
print("Wartość celu u(p*) =", u(p_opt))

Rozwiązanie p* = [0. 1. 0.]
Suma p* = 1.0
Wartość celu u(p*) = 0.34366925127106374
